In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings

using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
# include("Models_PLD.jl")
# include("Evaluation_PLD.jl")

estimate_parameters (generic function with 1 method)

In [26]:
N = 2 # num of product
N_x = 20 # num of product feature
c_l = ones(N_x)  # X * c_l >= d_r
d_r = ones(N) * 5
rev_gap = 0.001
N_u = 1 # num of customer feature
S_train = 200 # num of training samples
S_test = 1 # num of training samples
m = 5 # num of candidates in training samples
N_nonzero = 5 # num of nonzero entries in A
lambda_list = [0.01]
gamma_list = [0.0,0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.5]
gamma_list = [0.0]

instances = 10
Random.seed!(2)
project_dir = "N=$(N)_N_x=$(N_x)_N_u=$(N_u)_S_train=$(S_train)_N_nonzero=$(N_nonzero)/"

"N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/"

In [27]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")
if !isdir(data_dir)
    mkpath(data_dir)
end
data_dir = string(data_dir,project_dir)
println("Data directory: ", data_dir)

Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/


#### Data Generation

In [28]:
function compute_w(alpha0,alpha,beta,A,z_input)
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [29]:
Input_Data = Dict()
ins = 1
while ins <= instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    # ****** Data ******
    theta_true, r_params = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero);
    X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_train, m,theta_true);
    X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_test, m,theta_true);

    Input_Data["theta_true_ins=$ins"] = theta_true
    Input_Data["r_params_ins=$ins"] = r_params
    Input_Data["X_train_ins=$ins"] = X_train
    Input_Data["Y_train_ins=$ins"] = Y_train
    Input_Data["Z_train_ins=$ins"] = Z_train
    Input_Data["X_test_ins=$ins"] = X_test
    Input_Data["Y_test_ins=$ins"] = Y_test
    Input_Data["Z_test_ins=$ins"] = Z_test
    
    ins = ins + 1
    println(data_dir_ins)
end
save(string(data_dir,"Input_Data.jld2"),Input_Data)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=1/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=2/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=3/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=4/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=5/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=6/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=7/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=1_S_train=200_N_nonzero=5/instance=8/
/Users/zhangxun/Dropbox/

In [30]:


# ------------------------------------------------------------
# 1. 预计算所有扩展设计矩阵
# ------------------------------------------------------------
function precompute_extended_designs(X::Vector{Matrix{Float64}}, Z::Matrix{Float64})
    n = length(X)
    d = size(X[1], 2)
    p = size(Z, 2)
    total_dim = (d + 1) * (p + 1)
    X_tilde = Vector{Matrix{Float64}}(undef, n)

    # 预分配临时向量以避免重复分配
    x_tilde_temp = Vector{Float64}(undef, total_dim)

    for i in 1:n
        mi = size(X[i], 1)
        Xt = Matrix{Float64}(undef, mi, total_dim)

        @inbounds for j in 1:mi
            x_ij = @view X[i][j, :]
            z_i = @view Z[i, :]

            idx = 1
            x_tilde_temp[idx] = 1.0
            idx += 1

            @views x_tilde_temp[idx:idx+d-1] .= x_ij
            idx += d

            @inbounds for k in 1:p
                zk = z_i[k]
                x_tilde_temp[idx] = zk
                idx += 1
                @views x_tilde_temp[idx:idx+d-1] .= zk .* x_ij
                idx += d
            end

            @views Xt[j, :] .= x_tilde_temp
        end
        X_tilde[i] = Xt
    end
    return X_tilde
end

# ------------------------------------------------------------
# 2. 快速单样本负对数似然（使用预计算的 Xt_i）
# ------------------------------------------------------------
function neg_log_likelihood_single_fast(theta::AbstractVector, Xt_i::AbstractMatrix, y_i::Int)
    mi = size(Xt_i, 1)
    # 计算所有产品的效用：<θ, x̃_ij> for j=1..mi
    utilities = Xt_i * theta          # mi 维向量
    # 添加默认选项（效用为 0）
    m_val = maximum(utilities)
    # 数值稳定的 log-sum-exp: log(sum(exp(u))) = m + log(sum(exp(u - m)))
    shifted_utils = utilities .- m_val
    sum_exp = sum(exp, shifted_utils) + exp(-m_val)  # + exp(0 - m_val) for default option
    log_denominator = m_val + log(sum_exp)

    if y_i == 0
        log_prob = -log_denominator  # 因为 U0 = 0 → log(exp(0)/denom) = -log_denom
    else
        log_prob = utilities[y_i] - log_denominator
    end

    return -log_prob
end

# ------------------------------------------------------------
# 3. 快速整体负对数似然
# ------------------------------------------------------------
function neg_log_likelihood_fast(theta::AbstractVector, X_tilde::Vector{Matrix{Float64}}, Y::Vector{Int})
    n = length(Y)
    total_neg_ll = 0.0
    @inbounds for i in 1:n
        total_neg_ll += neg_log_likelihood_single_fast(theta, X_tilde[i], Y[i])
    end
    return total_neg_ll / n
end

# ------------------------------------------------------------
# 4. Lasso 目标函数（使用预计算数据）
# ------------------------------------------------------------
function lasso_objective_fast(theta::AbstractVector, lambda::Float64, X_tilde::Vector{Matrix{Float64}}, Y::Vector{Int})
    nll = neg_log_likelihood_fast(theta, X_tilde, Y)
    l1_penalty = lambda * norm(theta, 1)
    return nll + l1_penalty
end

# ------------------------------------------------------------
# 5. 参数解析（不变）
# ------------------------------------------------------------
function parameter_divide(theta_hat, d::Int, p::Int)
    alpha0_hat = 0.0
    alpha_hat = zeros(d)
    beta_hat = zeros(p)
    A_hat = zeros(d, p)

    idx = 1
    alpha0_hat = theta_hat[idx]
    idx += 1

    alpha_hat = theta_hat[idx:idx+d-1]
    idx += d

    for k in 1:p
        beta_hat[k] = theta_hat[idx]
        idx += 1
        A_hat[:, k] = theta_hat[idx:idx+d-1]
        idx += d
    end
    return alpha0_hat, alpha_hat, beta_hat, A_hat
end

# ------------------------------------------------------------
# 6. 主估计函数（优化版）
# ------------------------------------------------------------
function estimate_parameters_fast(
    X::Vector{Matrix{Float64}},
    Y::Vector{Int64},
    Z::Matrix{Float64},
    lambda::Float64,
    d::Int,
    p::Int;
    initial_theta = nothing,
    precomputed_X_tilde = nothing
)
    # 预计算扩展设计（如果未提供）
    if isnothing(precomputed_X_tilde)
        @time X_tilde = precompute_extended_designs(X, Z)
    else
        X_tilde = precomputed_X_tilde
    end

    total_dim = (d + 1) * (p + 1)
    if isnothing(initial_theta)
        initial_theta = randn(total_dim) * 0.1
    end

    # 目标函数闭包
    obj(theta) = lasso_objective_fast(theta, lambda, X_tilde, Y)

    # 使用 OWLQN 优化（专为 L1 设计）
    # result = optimize(
    #     obj,
    #     initial_theta,
    #     OWLQN(lambda),
    #     Optim.Options(show_trace = false, g_tol = 1e-6, iterations = 1000, time_limit = 300)
    # )
    result = optimize(obj, initial_theta, LBFGS(), Optim.Options(show_trace=false, g_tol=1e-6, iterations=1000,time_limit = 300))

    theta_hat = Optim.minimizer(result)
    alpha0_hat, alpha_hat, beta_hat, A_hat = parameter_divide(theta_hat, d, p)

    return alpha0_hat, alpha_hat, beta_hat, A_hat, result, X_tilde
end

estimate_parameters_fast (generic function with 1 method)

In [31]:
function compute_w(alpha0,alpha,beta,A,z_input)
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [32]:
lambda_list = [0.001]

1-element Vector{Float64}:
 0.001

In [33]:
for ins in 1:instances
    theta_true = Input_Data["theta_true_ins=$ins"]
    r_params = Input_Data["r_params_ins=$ins"]
    X_train = Input_Data["X_train_ins=$ins"]
    Y_train = Input_Data["Y_train_ins=$ins"]
    Z_train = Input_Data["Z_train_ins=$ins"]
    X_test = Input_Data["X_test_ins=$ins"]
    Y_test = Input_Data["Y_test_ins=$ins"]
    Z_test = Input_Data["Z_test_ins=$ins"]

    z_input = Z_test[1,:];
    nu0_true,nu_true = compute_w(theta_true.alpha0,theta_true.alpha,theta_true.beta,theta_true.A,z_input);
    w_true = [nu0_true;nu_true];
    println("w_true = ",round.(w_true,digits=4))
    for lambda in lambda_list
        alpha0, alpha, beta, A, result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, lambda, N_x, N_u);
        nu0_,nu_ = compute_w(alpha0,alpha,beta,A,z_input);
        w_ = [nu0_;nu_];
        println("w_ = ",round.(w_,digits=4))
        println("*** norm = ",norm(w_ .- w_true,2),"*****")
    end
    println("------------")
end

w_true = [1.2489, 0.4442, -0.295, -0.3563, -1.3427, 0.7667, 0.0998, 0.7176, -0.2489, -0.4407, -0.7148, 0.094, 0.4154, 1.1798, -1.7647, 0.9052, 0.0207, 0.7518, 0.161, 0.1593, -0.6618]
  0.000122 seconds (202 allocations: 355.281 KiB)
w_ = [1.3377, 0.838, -0.3362, -0.765, -0.9344, 0.5413, -0.0662, 0.6018, -0.0311, -0.694, -0.7322, 0.3736, 0.0932, 0.8604, -1.6579, 0.028, -0.0778, 0.7486, 0.1068, 0.1427, -0.2321]
*** norm = 1.4016363932812217*****
------------
w_true = [0.7601, -0.9555, 0.5936, 0.0002, -0.3098, -0.9214, 0.2334, -0.8233, -0.4618, -0.4877, -1.3791, -0.9112, 0.2202, 0.1884, -0.9203, 2.201, 0.0117, -0.4393, -0.2038, 0.215, -0.2797]
  0.000199 seconds (202 allocations: 355.281 KiB)
w_ = [2.1202, -1.4785, 0.3432, 0.0091, -0.181, -1.4591, 0.4873, -1.2157, -0.3854, -1.1873, -1.7161, -1.1473, 0.5062, 0.2277, -1.3423, 2.3558, 0.1045, -0.2182, -0.4529, 0.1495, -0.5001]
*** norm = 1.95775283909609*****
------------
w_true = [2.5547, -0.2549, 0.6046, 0.4216, -1.4614, 1.3551, 0.1317, 0.